# [TUTORIAL] Constrained Task Planning (Sweep)

This tutorial is written to instruct basic usage of the task & motion planning pipeline  
One Indy7 robot and several environment geometries will be added and floor-wiping task will be conducted.  
Here, the wiping task is defined as 1) contact with floor, 2) starting waypoint, 3) goal waypoint.  
Thus, any motion that satisfies the constraint will be generated; it may not look like real wiping.  

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection_list
[False, False]


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [5]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.3,0.5,0.01), (0.4,0,-0.005), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
floor_col = gscene.create_safe(GEOTYPE.BOX, "floor_col", "base_link", (0.3,0.5,0.01), (0.4,0,-0.006), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=False, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

Please create a subscriber to the marker
   Use a production WSGI server instead.
 * Debug mode: off


In [6]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add wp

In [7]:
wp11 = gscene.create_safe(GEOTYPE.BOX, "wp11", "base_link", (0.08,0.08,0.01), (0.5,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp12 = gscene.create_safe(GEOTYPE.BOX, "wp12", "base_link", (0.08,0.08,0.01), (0.5,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line1 = gscene.create_safe(GEOTYPE.BOX, "wline1", "base_link", (0.01,0.5,1e-6), (0.5,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp21 = gscene.create_safe(GEOTYPE.BOX, "wp21", "base_link", (0.08,0.08,0.01), (0.4,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp22 = gscene.create_safe(GEOTYPE.BOX, "wp22", "base_link", (0.08,0.08,0.01), (0.4,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line2 = gscene.create_safe(GEOTYPE.BOX, "wline2", "base_link", (0.01,0.5,1e-6), (0.4,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp31 = gscene.create_safe(GEOTYPE.BOX, "wp31", "base_link", (0.08,0.08,0.01), (0.3,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp32 = gscene.create_safe(GEOTYPE.BOX, "wp32", "base_link", (0.08,0.08,0.01), (0.3,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line3 = gscene.create_safe(GEOTYPE.BOX, "wline3", "base_link", (0.01,0.5,1e-6), (0.3,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

## add brush

In [8]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="indy0_tcp", dims=(0.1,0.07,0.02), 
                   center=(0.0,0.0,0.14), rpy=(0,np.pi,0), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=True)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="indy0_tcp", dims=(0.1,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=True, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face_col", link_name="indy0_tcp", dims=(0.1,0.06,0.028), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=True, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="indy0_tcp", dims=(0.1,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=True, parent="brush_body")

## add box

In [9]:
gbox1 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box1", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.5,0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

# gbox2 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box2", link_name="base_link", dims=(0.05,0.05,0.05), 
#                    center=(0.4,-0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool, FixtureSlot

In [11]:
# gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
#                  dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
# pscene.create_binder(bname="grip0", gname="grip0", rname="indy0", _type=FixtureSlot, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="grip1", gname="grip1", rname="panda1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="brush_face", gname="brush_face", rname="indy0", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [13]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask, BoxObject, FixturePoint, AbstractObject

In [14]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
# brush = pscene.create_object(oname="brush", gname="brush_body", _type=CustomObject, 
#                              action_points_dict = {"handle": FixturePoint("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
#                                                    "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [15]:
box1 = pscene.create_object(oname="box1", gname="box1", _type=BoxObject, hexahedral=True)
# box2 = pscene.create_object(oname="box2", gname="box2", _type=BoxObject, hexahedral=True)

In [16]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [17]:
sweep1 = pscene.create_object(oname="sweep1", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp11": SweepPoint("wp11", wp11, [0,0,0.005], [0,0,0]),
                                                   "wp12": SweepPoint("wp12", wp12, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line1)
sweep2 = pscene.create_object(oname="sweep2", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp21": SweepPoint("wp21", wp21, [0,0,0.005], [0,0,0]),
                                                   "wp22": SweepPoint("wp22", wp22, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line2)
sweep3 = pscene.create_object(oname="sweep3", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp31": SweepPoint("wp31", wp31, [0,0,0.005], [0,0,0]),
                                                   "wp32": SweepPoint("wp32", wp32, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line3)

### planners

In [18]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

## motion filters

In [19]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker

gcheck = GraspChecker(pscene, 
                      end_link_couple_dict= {
                          "indy0_tcp": ["indy0_tcp", "indy0_link6"],
                          "panda1_hand": ["panda1_hand", "panda1_link6"],
                          "base_link":["base_link"]})
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, end_link_couple_dict= {TOOL_LINK: TOOL_LINK_BUNDLE, "base_link":["base_link"]})
# checkers_all.append(lcheck)

In [20]:
mplan.motion_filters = checkers_all

In [21]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [22]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.update_state(crob.home_pose)
print(initial_state.node)

('floor', 0, 0, 0)


In [23]:
pscene.subject_name_list

['box1', 'sweep1', 'sweep2', 'sweep3']

# Node Sampler

In [24]:
from pkg.planning.sampling.node_sampling import NodeSampler

In [25]:
tplan.new_node_sampler = NodeSampler(0.5)
tplan.parent_node_sampler = NodeSampler(0.5)

# CustomRule

In [26]:
class CustomRule:
    def __init__(self, pscene):
        self.pscene = pscene
        self.chain_dict = pscene.get_robot_chain_dict()
        
    def __call__(self, tplan, snode_src, snode_new, connection_result):
        if snode_src is not None:
            diff_sidxes = np.where([ ntem_s != ntem_g for ntem_s, ntem_g in zip(snode_src.state.node, snode_new.state.node)])[0]
            if len(diff_sidxes)==0:
                return False, None
            diff_sidx = diff_sidxes[0]
            diff_sname = pscene.subject_name_list[diff_sidx]
            diff_subject = pscene.subject_dict[diff_sname]
            if isinstance(diff_subject, AbstractObject):
                link_name1 = snode_src.state.state_param[diff_sname][0]
                link_name2 = snode_new.state.state_param[diff_sname][0]
                rname_candis = [rname for rname, chain_vals in self.chain_dict.items() if 
                               link_name1 in chain_vals['link_names'] or link_name2 in chain_vals['link_names']]
                if len(rname_candis)==0:
                    print("no robot candis")
                    return False, None
                else:
                    newstate = snode_new.state.copy(self.pscene)
                    jidxes = self.pscene.combined_robot.idx_dict[rname_candis[0]]
                    newstate.Q[jidxes] = self.pscene.combined_robot.home_pose[jidxes]
                    print("try: {}".format(newstate.node))
                    return True, newstate
            else:
                return False, None
        else:
            return False, None

In [27]:
tplan.custom_rule = CustomRule(pscene)

In [28]:
gtimer.reset()
goal_nodes = [("floor", 2, 2, 2)]
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=True, timeout=1, timeout_constrained=1)
gtimer.toc("plan")
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

Use 20/20 agents
=============== try extend to goal ('floor', 0, 0, 0) -> ('floor', 0, 1, 0) =================
node: ('floor', 0, 0, 0)->('floor', 0, 0, 0) = fail
node: ('floor', 0, 0, 0)->('floor', 0, 0, 0) = fail
node: ('floor', 0, 0, 0)->('floor', 0, 0, 0) = fail
node: ('floor', 0, 0, 0)->('floor', 0, 1, 0) = success
branching: 0->1 (0.08/300.0 s, steps/err: 11(39.8390293121 ms)/0.00128817324591)
=============== try extend to goal ('floor', 0, 0, 1) -> ('floor', 0, 0, 2) =================
node: ('floor', 0, 0, 0)->('floor', 0, 0, 1) = success
branching: 0->2 (0.16/300.0 s, steps/err: 11(61.0840320587 ms)/0.00188889847546)
=============== try extend to goal ('floor', 0, 1, 0) -> ('floor', 0, 2, 0) =================
node: ('floor', 0, 0, 0)->('floor', 0, 1, 0) = success
branching: 0->4 (0.18/300.0 s, steps/err: 16(75.6649971008 ms)/0.00127615461873)
=============== try extend to goal ('floor', 0, 0, 1) -> ('floor', 0, 0, 2) =================
node: ('floor', 0, 0, 0)->('floor', 0, 0, 1

=============== try extend to goal ('floor', 0, 0, 1) -> ('floor', 0, 0, 2) =================
node: ('floor', 1, 0, 0)->('floor', 2, 0, 0) = fail
node: ('floor', 0, 0, 0)->('floor', 0, 0, 1) = success
branching: 0->21 (1.72/300.0 s, steps/err: 19(128.378152847 ms)/0.000774891662476)
node: ('floor', 0, 0, 1)->('floor', 0, 0, 2) = fail
node: ('grip1', 0, 0, 0)->('grip1', 0, 0, 0) = fail
node: ('floor', 0, 1, 0)->('floor', 0, 2, 0) = fail
node: ('grip1', 0, 0, 0)->('grip1', 0, 1, 0) = success
branching: 12->22 (2.2/300.0 s, steps/err: 10(67.7289962769 ms)/0.00174873640826)
=============== try extend to goal ('grip1', 2, 0, 0) -> ('grip1', 2, 0, 1) =================
node: ('grip1', 1, 0, 0)->('grip1', 2, 0, 0) = success
branching: 18->23 (2.37/300.0 s, steps/err: 68(857.336997986 ms)/0.00139983102635)
node: ('grip1', 0, 1, 0)->('grip1', 0, 2, 0) = fail
node: ('floor', 0, 1, 0)->('floor', 0, 2, 0) = fail
node: ('grip1', 2, 0, 0)->('grip1', 2, 0, 1) = success
branching: 23->24 (2.48/300.0 s,

=============== try extend to goal ('grip1', 2, 2, 1) -> ('grip1', 2, 2, 2) =================
node: ('grip1', 2, 2, 0)->('grip1', 2, 2, 1) = success
branching: 37->41 (3.18/300.0 s, steps/err: 9(46.972990036 ms)/0.0014819983274)
node: ('grip1', 0, 0, 0)->('grip1', 1, 0, 0) = fail
node: ('grip1', 0, 0, 0)->('grip1', 0, 1, 0) = success
branching: 12->42 (3.31/300.0 s, steps/err: 26(60.8592033386 ms)/0.00143518856803)
node: ('grip1', 2, 1, 0)->('grip1', 2, 2, 0) = fail
node: ('grip1', 2, 0, 1)->('grip1', 2, 0, 2) = fail
try: ('floor', 2, 2, 0)
node: ('grip1', 2, 0, 1)->('grip1', 2, 0, 2) = fail
node: ('grip1', 2, 2, 0)->('floor', 2, 2, 0) = success
branching: 37->43 (3.67/300.0 s, steps/err: 24(328.905105591 ms)/0.00177151396516)
node: ('grip1', 1, 0, 0)->('grip1', 2, 0, 0) = fail
============= try go home (panda1) ===================
node: ('grip1', 0, 0, 1)->('grip1', 0, 0, 2) = fail
node: ('grip1', 2, 1, 0)->('grip1', 2, 2, 0) = fail
=============== try extend to goal ('grip1', 2, 0, 1

node: ('grip1', 2, 1, 0)->('grip1', 2, 2, 0) = fail
node: ('grip1', 1, 0, 0)->('grip1', 2, 0, 0) = fail
node: ('grip1', 2, 2, 1)->('grip1', 2, 2, 2) = fail
=============== try extend to goal ('grip1', 2, 1, 0) -> ('grip1', 2, 2, 0) =================
node: ('grip1', 2, 2, 1)->('grip1', 2, 2, 2) = fail
node: ('grip1', 2, 0, 0)->('grip1', 2, 1, 0) = success
node: ('grip1', 0, 0, 0)->('grip1', 0, 0, 1) = success
branching: 46->61 (5.11/300.0 s, steps/err: 11(75.2799510956 ms)/0.00160285204189)
branching: 35->60 (5.1/300.0 s, steps/err: 30(59.1208934784 ms)/0.00106913439701)
============= try go home (indy0) ===================
node: ('floor', 2, 2, 0)->('floor', 2, 2, 1) = fail
node: ('grip1', 0, 0, 0)->('grip1', 0, 1, 0) = success
branching: 46->62 (5.16/300.0 s, steps/err: 11(64.7599697113 ms)/0.00138619243066)
node: ('grip1', 2, 2, 1)->('grip1', 2, 2, 2) = fail
=============== try extend to goal ('floor', 2, 2, 0) -> ('floor', 2, 2, 0) =================
node: ('floor', 2, 2, 0)->('floor

node: ('floor', 0, 2, 0)->('floor', 0, 2, 1) = success
branching: 75->78 (6.55/300.0 s, steps/err: 9(69.4451332092 ms)/0.000815665640901)
=============== try extend to goal ('floor', 1, 2, 0) -> ('floor', 2, 2, 0) =================
node: ('grip1', 0, 0, 0)->('grip1', 0, 0, 1) = success
node: ('floor', 2, 2, 1)->('floor', 2, 2, 2) = fail
node: ('floor', 0, 2, 0)->('floor', 1, 2, 0) = success
branching: 12->79 (6.61/300.0 s, steps/err: 14(218.08385849 ms)/0.00191730301188)
branching: 75->80 (6.61/300.0 s, steps/err: 23(122.903823853 ms)/0.00167971307786)
=============== try extend to goal ('floor', 1, 2, 0) -> ('floor', 2, 2, 0) =================
node: ('floor', 0, 2, 0)->('floor', 1, 2, 0) = success
branching: 75->81 (6.63/300.0 s, steps/err: 15(74.1801261902 ms)/0.0017861760723)
=============== try reaching goal from ('grip1', 2, 2, 2) =================
node: ('grip1', 2, 2, 1)->('grip1', 2, 2, 2) = success
branching: 76->82 (6.68/300.0 s, steps/err: 85(232.191085815 ms)/0.001913929635

...

node: ('grip1', 0, 2, 0)->('grip1', 1, 2, 0) = success
branching: 175->194 (14.19/300.0 s, steps/err: 13(66.055059433 ms)/0.00175122220663)
=============== try extend to goal ('grip1', 2, 2, 1) -> ('grip1', 2, 2, 2) =================
node: ('grip1', 2, 2, 0)->('grip1', 2, 2, 1) = success
branching: 37->195 (14.22/300.0 s, steps/err: 8(76.7800807953 ms)/0.00159057054081)
node: ('grip1', 1, 2, 0)->('grip1', 2, 2, 0) = fail
=============== try extend to goal ('grip1', 1, 2, 0) -> ('grip1', 2, 2, 0) =================
=============== try reaching goal from ('floor', 2, 2, 1) =================
node: ('grip1', 0, 2, 0)->('grip1', 1, 2, 0) = success
node: ('grip1', 0, 2, 1)->('grip1', 0, 2, 2) = fail
branching: 175->197 (14.36/300.0 s, steps/err: 9(71.2440013885 ms)/0.0012285810512)
node: ('floor', 2, 2, 0)->('floor', 2, 2, 1) = success
branching: 50->196 (14.37/300.0 s, steps/err: 14(68.9690113068 ms)/0.00208519805669)
node: ('grip1', 1, 2, 0)->('grip1', 2, 2, 0) = fail
node: ('grip1', 0, 2, 

node: ('floor', 0, 0, 0)->('floor', 0, 1, 0) = success
branching: 0->214 (15.09/300.0 s, steps/err: 23(89.7371768951 ms)/0.00184009038955)
node: ('grip1', 0, 0, 0)->('grip1', 1, 0, 0) = success
branching: 38->215 (15.19/300.0 s, steps/err: 12(33.1320762634 ms)/0.00144083915362)
node: ('grip1', 2, 2, 1)->('grip1', 2, 2, 2) = fail
node: ('grip1', 1, 2, 0)->('grip1', 2, 2, 0) = fail
=============== try extend to goal ('grip1', 2, 1, 0) -> ('grip1', 2, 2, 0) =================
node: ('grip1', 2, 0, 0)->('grip1', 2, 1, 0) = success
branching: 35->216 (15.46/300.0 s, steps/err: 11(68.0780410767 ms)/0.00115542246082)
node: ('floor', 2, 2, 1)->('floor', 2, 2, 2) = fail
=============== try extend to goal ('grip1', 2, 2, 1) -> ('grip1', 2, 2, 2) =================
node: ('grip1', 2, 2, 0)->('grip1', 2, 2, 1) = success
branching: 199->217 (15.58/300.0 s, steps/err: 14(74.2330551147 ms)/0.00140016863766)
node: ('grip1', 0, 2, 1)->('grip1', 0, 2, 2) = fail
node: ('floor', 1, 2, 0)->('floor', 2, 2, 0)

branching: 219->233 (16.38/300.0 s, steps/err: 18(96.7769622803 ms)/0.00199920543955)
branching: 12->234 (16.37/300.0 s, steps/err: 27(81.5830230713 ms)/0.00145549992803)
node: ('grip1', 0, 0, 0)->('grip1', 0, 1, 0) = success
branching: 38->235 (16.48/300.0 s, steps/err: 11(50.3611564636 ms)/0.0013677157229)
=============== try extend to goal ('grip1', 2, 0, 1) -> ('grip1', 2, 0, 2) =================
=============== try extend to goal ('floor', 0, 0, 0) -> ('floor', 0, 0, 1) =================
node: ('floor', 0, 0, 0)->('floor', 0, 0, 0) = success
node: ('grip1', 2, 0, 0)->('grip1', 2, 0, 1) = success
branching: 159->237 (16.51/300.0 s, steps/err: 34(203.543186188 ms)/8.01646793915e-16)
branching: 219->236 (16.51/300.0 s, steps/err: 10(72.9019641876 ms)/0.00142562519373)
node: ('grip1', 2, 1, 0)->('grip1', 2, 2, 0) = fail
node: ('floor', 0, 0, 0)->('floor', 0, 1, 0) = fail
=============== try reaching goal from ('floor', 2, 2, 1) =================
node: ('floor', 0, 0, 0)->('floor', 0, 

node: ('grip1', 2, 0, 1)->('grip1', 2, 0, 2) = fail
node: ('floor', 0, 2, 2)->('floor', 1, 2, 2) = fail
node: ('floor', 0, 2, 2)->('floor', 1, 2, 2) = fail
node: ('floor', 2, 2, 1)->('floor', 2, 2, 2) = fail
node: ('floor', 0, 1, 0)->('floor', 0, 2, 0) = fail
=============== try extend to goal ('floor', 1, 2, 0) -> ('floor', 2, 2, 0) =================
node: ('floor', 0, 2, 0)->('floor', 1, 2, 0) = success
try: ('floor', 0, 2, 0)
branching: 75->255 (17.82/300.0 s, steps/err: 18(111.752033234 ms)/0.00179471211049)
node: ('grip1', 0, 2, 0)->('floor', 0, 2, 0) = success
node: ('grip1', 1, 2, 0)->('grip1', 2, 2, 0) = fail
=============== try reaching goal from ('floor', 2, 2, 1) =================
branching: 244->256 (17.86/300.0 s, steps/err: 30(699.955940247 ms)/0.000891617222153)
node: ('floor', 0, 1, 0)->('floor', 0, 2, 0) = fail
node: ('floor', 2, 2, 0)->('floor', 2, 2, 1) = success
branching: 49->257 (17.9/300.0 s, steps/err: 26(109.82298851 ms)/0.00238727317008)
node: ('floor', 0, 0, 

branching: 110->273 (19.32/300.0 s, steps/err: 34(229.858875275 ms)/0.000908698510263)
node: ('grip1', 0, 0, 0)->('grip1', 0, 1, 0) = success
branching: 38->274 (19.39/300.0 s, steps/err: 14(59.2131614685 ms)/0.00115608315114)
======================= terminated 0: first answer acquired ===============================
======================= terminated 6: first answer acquired from other agent ===============================
node: ('grip1', 0, 1, 0)->('grip1', 0, 2, 0) = fail
======================= terminated 7: first answer acquired from other agent ===============================
node: ('grip1', 2, 2, 1)->('grip1', 2, 2, 2) = fail
======================= terminated 19: first answer acquired from other agent ===============================
node: ('floor', 0, 0, 1)->('floor', 0, 0, 2) = fail
node: ('floor', 1, 2, 0)->('floor', 2, 2, 0) = fail
======================= terminated 18: first answer acquired from other agent ===============================
node: ('floor', 0, 2, 0)->('floor',

In [29]:
print(gtimer)

plan: 	19385.0 ms/1 = 19385.216 ms (19385.216/19385.216)



## play searched plan

In [30]:
ppline.play_schedule(snode_schedule[:2], period=0.01)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [31]:
snode_schedule = ppline.add_return_motion(snode_schedule)

In [32]:
ppline.play_schedule(snode_schedule, period=0.03)

In [33]:
print("schedule length: {}".format(len(snode_schedule)))
for i_s,  snode in enumerate(snode_schedule):
    print("{}: {}".format(i_s, snode.state.node))

schedule length: 12
0: ('floor', 0, 0, 0)
1: ('grip1', 0, 0, 0)
2: ('grip1', 0, 0, 0)
3: ('grip1', 1, 0, 0)
4: ('grip1', 2, 0, 0)
5: ('grip1', 2, 1, 0)
6: ('grip1', 2, 2, 0)
7: ('grip1', 2, 2, 1)
8: ('grip1', 2, 2, 2)
9: ('grip1', 2, 2, 2)
10: ('floor', 2, 2, 2)
11: ('floor', 2, 2, 2)


## NOTE
* 위에 가운데 열 어떻게  0,1,1,1,1,2가 나오지? -> 일방통행 작업 플래그 추가
* 균등 샘플링 - 샘플 할때마다 노드별/전환별 확률 조정
* goal-directed extension 추가.

In [56]:
tplan.node_dict[('grip0', 'goal', 0, 1, 2)]

{('floor', 'goal', 0, 1, 2),
 ('goal', 'goal', 0, 1, 2),
 ('grip0', 'goal', 0, 2, 2),
 ('grip0', 'goal', 1, 1, 2)}

## extend preserving goal-matching items

In [25]:
print(gtimer)

plan: 	207291.0 ms/10 = 20729.062 ms (9041.162/45383.357)



## extend only no reservation

In [25]:
print(gtimer)

plan: 	270954.0 ms/10 = 27095.38 ms (6255.591/68399.08)



## no extension

In [25]:
print(gtimer)

plan: 	226519.0 ms/10 = 22651.91 ms (8120.631/69353.952)



## extend_toward goal

In [30]:
print(gtimer)

plan: 	204994.0 ms/10 = 20499.448 ms (8105.979/60408.919)

